In [181]:
import numpy as np
import statsmodels.tsa.stattools as ts 
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import pandas as pd
import wrds

In [230]:
# Login
import wrds
db = wrds.Connection(wrds_username="hanyuzhang")
# password is Timeseries2020!

Enter your WRDS username [liyf4]:hanyuzhang
Enter your password:········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [231]:
# Preprocess Data
import pandas as pd
sym_df = pd.read_csv("Ticker.csv", header=0)
sym_list = sym_df["Ticker"].tolist()
sym_permno_list = list()
for symbol in sym_list:
    if len(symbol) == 0:
        continue
    result = db.raw_sql(f"""select permno, htsymbol 
                           from crsp.dsfhdr 
                           where htsymbol = '{symbol}'""")
    try:
        sym_permno_list.append((result.iloc[0]['permno'], result.iloc[0]['htsymbol']))
    except:
        print(f"wrds doesn't have data for {symbol} right now, skip...")

wrds doesn't have data for ATT right now, skip...
wrds doesn't have data for RTX right now, skip...


In [184]:
from collections import defaultdict
timeseries_bank = defaultdict(list)
for permno, symbol in sym_permno_list:
    result = db.raw_sql(f"""select date, prc, cfacpr
                            from crsp.dsf
                            where permno = {permno} and date > '2010-01-01'
                         """)
    ts = {date:prc for date, prc in zip(result["date"].tolist(), (result["prc"]/result["cfacpr"]).tolist())}
    timeseries_bank[symbol] = ts  

In [185]:

# Build pairs
sym_pairs = list()
sym_pairs_ts = dict()
valid_sym_list = list(timeseries_bank.keys())
for i in range(len(valid_sym_list)-1):
    for j in range(i+1, len(valid_sym_list)):
        sym_pairs.append((valid_sym_list[i], valid_sym_list[j]))
for pair in sym_pairs:
    sym_pairs_ts[pair] = {"first":timeseries_bank[pair[0]], "second":timeseries_bank[pair[1]]}
    
# sym_paris_ts is the pair timeseries:
# {(sym1, sym2):{'first':[(datetime.date(2010,1,1),price1), (datetime.date(2010,1,2):price2), ...}, "second":[...]}}

In [24]:
raw_data=pd.DataFrame()
for key,value in timeseries_bank.items():
    raw_data[key]=pd.Series(np.array(list(value.values())).flatten())

In [30]:
del raw_data["T"]

In [43]:
return_data=np.log(1+raw_data.pct_change().dropna())

In [76]:
return_data.cov()

,AAPL,MSFT,JPM,JNJ,XOM,BAC,WFC,WMT,UNH,CVX,...,UNP,TXN,LLY,KO,PG,PEP,GOOGL,VZ,BA,GS
AAPL,0.000264,0.000106,0.000095,0.000045,0.000068,0.000120,0.000090,0.000040,0.000076,0.000075,...,0.000093,0.000118,0.000050,0.000039,0.000039,0.000038,0.000115,0.000037,0.000099,0.000101
MSFT,0.000106,0.000205,0.000107,0.000054,0.000075,0.000125,0.000099,0.000048,0.000079,0.000083,...,0.000090,0.000118,0.000061,0.000049,0.000048,0.000049,0.000118,0.000048,0.000097,0.000109
JPM,0.000095,0.000107,0.000249,0.000065,0.000104,0.000268,0.000189,0.000051,0.000097,0.000113,...,0.000128,0.000122,0.000070,0.000053,0.000049,0.000049,0.000100,0.000059,0.000126,0.000207
JNJ,0.000045,0.000054,0.000065,0.000091,0.000053,0.000069,0.000062,0.000037,0.000055,0.000055,...,0.000056,0.000058,0.000058,0.000040,0.000041,0.000042,0.000052,0.000042,0.000062,0.000061
XOM,0.000068,0.000075,0.000104,0.000053,0.000139,0.000118,0.000098,0.000040,0.000068,0.000124,...,0.000091,0.000085,0.000053,0.000047,0.000045,0.000042,0.000072,0.000048,0.000088,0.000098
BAC,0.000120,0.000125,0.000268,0.000069,0.000118,0.000422,0.000235,0.000057,0.000119,0.000133,...,0.000157,0.000146,0.000077,0.000057,0.000050,0.000050,0.000125,0.000064,0.000154,0.000253
WFC,0.000090,0.000099,0.000189,0.000062,0.000098,0.000235,0.000230,0.000050,0.000094,0.000106,...,0.000124,0.000114,0.000065,0.000053,0.000047,0.000047,0.000095,0.000059,0.000123,0.000173
WMT,0.000040,0.000048,0.000051,0.000037,0.000040,0.000057,0.000050,0.000119,0.000044,0.000041,...,0.000045,0.000044,0.000038,0.000037,0.000040,0.000039,0.000040,0.000039,0.000050,0.000048
UNH,0.000076,0.000079,0.000097,0.000055,0.000068,0.000119,0.000094,0.000044,0.000206,0.000073,...,0.000082,0.000077,0.000064,0.000042,0.000037,0.000037,0.000078,0.000040,0.000086,0.000093
CVX,0.000075,0.000083,0.000113,0.000055,0.000124,0.000133,0.000106,0.000041,0.000073,0.000177,...,0.000104,0.000095,0.000059,0.000051,0.000046,0.000042,0.000075,0.000054,0.000095,0.000108


In [84]:
w,V=np.linalg.eig((return_data.iloc[-252:,:]-return_data.iloc[-252:,:].mean()).cov())

In [125]:
new_cov=V[:,:8].dot(np.diag(w[:8])).dot(V[:,:8].T)

In [126]:
return_data.iloc[-252:,:].corr()

,AAPL,MSFT,JPM,JNJ,XOM,BAC,WFC,WMT,UNH,CVX,...,UNP,TXN,LLY,KO,PG,PEP,GOOGL,VZ,BA,GS
AAPL,1.000000,0.621367,0.471701,0.254252,0.413924,0.436461,0.365538,0.175460,0.212553,0.360046,...,0.451438,0.551879,0.309024,0.162470,0.186099,0.317105,0.559352,0.073207,0.320116,0.478765
MSFT,0.621367,1.000000,0.461045,0.306938,0.397212,0.445891,0.404008,0.377300,0.295536,0.373949,...,0.429464,0.524543,0.326952,0.264600,0.327677,0.424078,0.636984,0.146254,0.352762,0.469184
JPM,0.471701,0.461045,1.000000,0.202456,0.552460,0.854018,0.673837,0.325515,0.224021,0.406388,...,0.567392,0.470756,0.153725,0.133134,0.139162,0.195665,0.422961,0.119240,0.310314,0.793497
JNJ,0.254252,0.306938,0.202456,1.000000,0.236920,0.140948,0.140719,0.173696,0.200298,0.173906,...,0.223140,0.185059,0.336814,0.182631,0.209275,0.337314,0.227554,0.210973,0.228781,0.145680
XOM,0.413924,0.397212,0.552460,0.236920,1.000000,0.523962,0.528053,0.238602,0.187933,0.737016,...,0.465912,0.404406,0.184538,0.106364,0.139911,0.228873,0.398222,0.169105,0.306702,0.487990
BAC,0.436461,0.445891,0.854018,0.140948,0.523962,1.000000,0.735879,0.279916,0.252037,0.407383,...,0.507644,0.450843,0.133691,0.080612,0.076671,0.076668,0.393581,0.050026,0.327403,0.857620
WFC,0.365538,0.404008,0.673837,0.140719,0.528053,0.735879,1.000000,0.223600,0.246142,0.425228,...,0.414459,0.395758,0.143533,0.097852,0.120820,0.074318,0.327811,0.098210,0.264129,0.674679
WMT,0.175460,0.377300,0.325515,0.173696,0.238602,0.279916,0.223600,1.000000,0.147015,0.188167,...,0.264144,0.209471,0.096142,0.241473,0.379459,0.429771,0.248440,0.257517,0.131436,0.297170
UNH,0.212553,0.295536,0.224021,0.200298,0.187933,0.252037,0.246142,0.147015,1.000000,0.207198,...,0.213069,0.131442,0.245349,0.059294,0.107191,0.130479,0.295841,0.147030,0.049165,0.227519
CVX,0.360046,0.373949,0.406388,0.173906,0.737016,0.407383,0.425228,0.188167,0.207198,1.000000,...,0.370882,0.399785,0.169693,0.119498,0.071404,0.145510,0.279358,0.156538,0.218092,0.401247


In [127]:
def correlation_from_covariance(covariance):
    v = np.sqrt(np.diag(covariance))
    outer_v = np.outer(v, v)
    correlation = covariance / outer_v
    correlation[covariance == 0] = 0
    return correlation

In [128]:
pd.DataFrame(correlation_from_covariance(new_cov))

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,1.000000,0.867306,0.575718,0.506065,0.726245,0.540039,0.470932,0.297613,0.282345,0.755131,...,0.711461,0.814993,0.447494,0.190322,0.230314,0.372668,0.834520,0.039220,0.387812,0.566828
1,0.867306,1.000000,0.589751,0.587441,0.652947,0.562376,0.533831,0.642629,0.353663,0.614452,...,0.654638,0.654292,0.565653,0.489819,0.536826,0.638423,0.913369,0.267871,0.407621,0.587253
2,0.575718,0.589751,1.000000,0.331781,0.922938,0.987792,0.965516,0.611947,0.323928,0.847858,...,0.901330,0.576241,0.132652,0.204652,0.249832,0.259690,0.545417,0.267150,0.360468,0.985387
3,0.506065,0.587441,0.331781,1.000000,0.586928,0.260972,0.336504,0.435526,0.457576,0.616806,...,0.503811,0.388547,0.861123,0.582416,0.607305,0.685606,0.388434,0.714868,0.490423,0.216865
4,0.726245,0.652947,0.922938,0.586928,1.000000,0.888985,0.883679,0.485274,0.374840,0.968718,...,0.938201,0.684429,0.370430,0.249857,0.281303,0.336514,0.570690,0.354820,0.478799,0.871679
5,0.540039,0.562376,0.987792,0.260972,0.888985,1.000000,0.976305,0.551094,0.372613,0.807067,...,0.850690,0.514469,0.099397,0.102586,0.147978,0.154384,0.546628,0.162088,0.371965,0.993928
6,0.470932,0.533831,0.965516,0.336504,0.883679,0.976305,1.000000,0.570678,0.424170,0.814927,...,0.861427,0.515990,0.222935,0.153649,0.183678,0.186269,0.461382,0.264543,0.339185,0.951422
7,0.297613,0.642629,0.611947,0.435526,0.485274,0.551094,0.570678,1.000000,0.135651,0.370703,...,0.553716,0.297982,0.280320,0.783818,0.831808,0.800181,0.492372,0.675627,0.176677,0.572039
8,0.282345,0.353663,0.323928,0.457576,0.374840,0.372613,0.424170,0.135651,1.000000,0.438815,...,0.212155,0.077092,0.608667,0.001176,0.035802,0.098076,0.432913,0.172041,-0.000495,0.328437
9,0.755131,0.614452,0.847858,0.616806,0.968718,0.807067,0.814927,0.370703,0.438815,1.000000,...,0.932779,0.770972,0.463722,0.163026,0.193947,0.278426,0.512038,0.317508,0.389896,0.782006


In [107]:
return_data.corr()

,AAPL,MSFT,JPM,JNJ,XOM,BAC,WFC,WMT,UNH,CVX,...,UNP,TXN,LLY,KO,PG,PEP,GOOGL,VZ,BA,GS
AAPL,1.000000,0.455324,0.371492,0.289657,0.353713,0.358661,0.364978,0.225897,0.323880,0.348275,...,0.390892,0.467751,0.244429,0.253764,0.254436,0.262082,0.465545,0.210376,0.387523,0.376540
MSFT,0.455324,1.000000,0.472139,0.397051,0.444400,0.426579,0.456539,0.306880,0.385707,0.436400,...,0.429090,0.534343,0.338090,0.366475,0.352554,0.378171,0.539150,0.315393,0.431056,0.460526
JPM,0.371492,0.472139,1.000000,0.430566,0.556720,0.828324,0.788444,0.297171,0.428490,0.537927,...,0.551723,0.497939,0.348234,0.357858,0.330657,0.345677,0.415466,0.351484,0.509393,0.797513
JNJ,0.289657,0.397051,0.430566,1.000000,0.470672,0.352073,0.430303,0.354634,0.400574,0.429799,...,0.397806,0.395113,0.478327,0.442816,0.457984,0.489157,0.355663,0.415998,0.413745,0.388410
XOM,0.353713,0.444400,0.556720,0.470672,1.000000,0.487393,0.546618,0.310875,0.402604,0.792505,...,0.524870,0.466793,0.356923,0.421784,0.402861,0.393679,0.399408,0.379180,0.474832,0.506810
BAC,0.358661,0.426579,0.828324,0.352073,0.487393,1.000000,0.754289,0.256500,0.404345,0.485523,...,0.519530,0.458929,0.295407,0.295361,0.259172,0.269358,0.397848,0.288902,0.478728,0.747300
WFC,0.364978,0.456539,0.788444,0.430303,0.546618,0.754289,1.000000,0.304312,0.430950,0.523708,...,0.555764,0.483887,0.336747,0.373509,0.326284,0.343427,0.411127,0.360914,0.516096,0.690908
WMT,0.225897,0.306880,0.297171,0.354634,0.310875,0.256500,0.304312,1.000000,0.284120,0.279621,...,0.282973,0.260734,0.275855,0.360697,0.388644,0.395524,0.242642,0.337447,0.294157,0.265119
UNH,0.323880,0.385707,0.428490,0.400574,0.402604,0.404345,0.430950,0.284120,1.000000,0.382680,...,0.387637,0.346510,0.350224,0.314547,0.277005,0.289821,0.357804,0.262860,0.380832,0.394395
CVX,0.348275,0.436400,0.537927,0.429799,0.792505,0.485523,0.523708,0.279621,0.382680,1.000000,...,0.529977,0.458910,0.352054,0.406632,0.364569,0.353463,0.368559,0.375371,0.452002,0.490579


In [118]:
sum(w[:8])/sum(w)

0.725760575059231

In [112]:
w[1]/sum(w)

0.09955169768643185

In [141]:
temp=np.array(list(timeseries_bank["AAPL"].keys())).flatten()
temp[(temp>datetime.date(2010,1,8)) &(temp<datetime.date(2012,1,8))]

array([datetime.date(2010, 1, 11), datetime.date(2010, 1, 12),
       datetime.date(2010, 1, 13), datetime.date(2010, 1, 14),
       datetime.date(2010, 1, 15), datetime.date(2010, 1, 19),
       datetime.date(2010, 1, 20), datetime.date(2010, 1, 21),
       datetime.date(2010, 1, 22), datetime.date(2010, 1, 25),
       datetime.date(2010, 1, 26), datetime.date(2010, 1, 27),
       datetime.date(2010, 1, 28), datetime.date(2010, 1, 29),
       datetime.date(2010, 2, 1), datetime.date(2010, 2, 2),
       datetime.date(2010, 2, 3), datetime.date(2010, 2, 4),
       datetime.date(2010, 2, 5), datetime.date(2010, 2, 8),
       datetime.date(2010, 2, 9), datetime.date(2010, 2, 10),
       datetime.date(2010, 2, 11), datetime.date(2010, 2, 12),
       datetime.date(2010, 2, 16), datetime.date(2010, 2, 17),
       datetime.date(2010, 2, 18), datetime.date(2010, 2, 19),
       datetime.date(2010, 2, 22), datetime.date(2010, 2, 23),
       datetime.date(2010, 2, 24), datetime.date(2010, 2, 25),

In [132]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
import datetime

In [187]:
len(sym_pairs_ts.keys())

406

In [188]:
def CointegrationTest(pairs,start_date,end_date,data):
    """
    pair: list of tuple
    date: datetime.date(YYYY,MM,DD)
    data: timeseries_bank
    """
    final_pairs=[]
    for pair in pairs:
        timestamp=np.array(list(data[pair[0]].keys())).flatten()
        #print(pair,len(timestamp),len(list(data[pair[0]].values())),len(list(data[pair[1]].values())))
        t1=np.log(np.array(list(data[pair[0]].values())).flatten()[(timestamp>=start_date) & (timestamp<=end_date)])
        t2=np.log(np.array(list(data[pair[1]].values())).flatten()[(timestamp>=start_date) & (timestamp<=end_date)])
        
        t1=t1/t1[0]
        t2=t2/t2[0]
        
        result=coint_johansen(np.array([t1,t2]).T, det_order=0, k_ar_diff=1)
        if np.all(result.lr1>=result.trace_stat_crit_vals[:,0]):
            final_pairs.append(pair)
    return final_pairs

In [189]:
listOfPairs=list(sym_pairs_ts.keys())

In [196]:
startDate=datetime.date(2012,10,1)
endDate=datetime.date(2013,10,1)
finalPair=CointegrationTest(listOfPairs,startDate,endDate,timeseries_bank)

In [226]:
func1=lambda w,V,numOfPC:V[:,:numOfPC].dot(np.diag(w[:numOfPC])).dot(V[:,:numOfPC].T)
func2=lambda w,V,numOfPC:V[:,:numOfPC].dot(np.diag(w[:numOfPC])).dot(V[:,:numOfPC].T)+\
                          sum(w[numOfPC:])/(len(w)-numOfPC)*(V[:,numOfPC:].dot(V[:,numOfPC:].T))


def CorrelationTest(pairs,start_date,end_date,data,numOfPC,func):
    """
    pair: list of tuple
    date: datetime.date(YYYY,MM,DD)
    data: timeseries_bank
    numOfCP: number of PC from evaluating correlation, we better has numOfPC<len(pairs)/2
    func: ways to calculate reduced corr
    """
    tickerList=[]
    for pair in pairs:
        tickerList=tickerList+list(pair)
    tickerList=list(set(tickerList))
    
    raw_data=pd.DataFrame()
    timestamp=np.array(list(data[tickerList[0]].keys())).flatten()
    dateRange=(timestamp>=start_date) & (timestamp<=end_date)
    for ticker in tickerList:
        temp=np.array(list(data[ticker].values())).flatten()[dateRange]
        raw_data[ticker]=pd.Series(temp)
    return_data=np.log(1+raw_data.pct_change().dropna())
    w,V=np.linalg.eig((return_data-return_data.mean()).cov())
    new_cov=func(w,V,numOfPC)
    new_corr=pd.DataFrame(correlation_from_covariance(new_cov),index=tickerList,columns=tickerList)
    ans={}
    for pair in pairs:
        ans[pair]=new_corr.loc[pair[0],pair[1]]
    return ans

In [227]:
startDate=datetime.date(2018,10,1)
endDate=datetime.date(2019,10,1)
correlation=CorrelationTest(listOfPairs,startDate,endDate,timeseries_bank,8,func2)

In [228]:
correlation

{('AAPL', 'MSFT'): 0.7007072290711909,
 ('AAPL', 'JPM'): 0.4962163322791779,
 ('AAPL', 'JNJ'): 0.3607896525563579,
 ('AAPL', 'XOM'): 0.5026938871201179,
 ('AAPL', 'BAC'): 0.5184924762068459,
 ('AAPL', 'WFC'): 0.4294017233248892,
 ('AAPL', 'WMT'): 0.33489278885767604,
 ('AAPL', 'UNH'): 0.448624291372542,
 ('AAPL', 'CVX'): 0.4954012609182795,
 ('AAPL', 'PFE'): 0.39577220901175303,
 ('AAPL', 'ADBE'): 0.7206489707131403,
 ('AAPL', 'MCD'): 0.2161632696876317,
 ('AAPL', 'MDT'): 0.49565320896530407,
 ('AAPL', 'MMM'): 0.5985681289735499,
 ('AAPL', 'HON'): 0.567541407886885,
 ('AAPL', 'GE'): 0.38041108868787316,
 ('AAPL', 'ABT'): 0.6210836710199634,
 ('AAPL', 'MO'): 0.29132554019108003,
 ('AAPL', 'UNP'): 0.555904672889221,
 ('AAPL', 'TXN'): 0.6920654470915758,
 ('AAPL', 'LLY'): 0.31149469942936014,
 ('AAPL', 'KO'): 0.185984586710926,
 ('AAPL', 'PG'): 0.21099299326227342,
 ('AAPL', 'PEP'): 0.2249526475587811,
 ('AAPL', 'GOOGL'): 0.6907192271489031,
 ('AAPL', 'VZ'): 0.0687710669352193,
 ('AAPL', 

In [220]:
len(correlation)

406